# **Restaurant Inspections New York City**

## **Imports**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
%matplotlib inline
from wordcloud import WordCloud
import re
import nltk
from nltk.corpus import stopwords

#Sklearn preprocessing
from sklearn import preprocessing,set_config
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder,StandardScaler,LabelEncoder
#Scipy
from scipy import stats
from scipy.stats import norm

#Sklearn Models
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn import preprocessing, set_config
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer,make_column_selector,make_column_transformer
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn.decomposition import PCA


# Warnings 
import warnings 
warnings.filterwarnings("ignore")

# Configuring diagrams
set_config(display = 'diagram')

# **Load Data**

In [2]:
rd = pd.read_csv('https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD')

# **Data Exploration**


In [3]:
# Check to see if data loaded properly:
rd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209789 entries, 0 to 209788
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  209789 non-null  int64  
 1   DBA                    209165 non-null  object 
 2   BORO                   209789 non-null  object 
 3   BUILDING               209481 non-null  object 
 4   STREET                 209783 non-null  object 
 5   ZIPCODE                207093 non-null  float64
 6   PHONE                  209781 non-null  object 
 7   CUISINE DESCRIPTION    207365 non-null  object 
 8   INSPECTION DATE        209789 non-null  object 
 9   ACTION                 207365 non-null  object 
 10  VIOLATION CODE         206208 non-null  object 
 11  VIOLATION DESCRIPTION  206208 non-null  object 
 12  CRITICAL FLAG          209789 non-null  object 
 13  SCORE                  199793 non-null  float64
 14  GRADE                  102961 non-nu

## **Drop Columns**

In [4]:
# Dropping irrelevant of imcomplete columns
rd.drop(['Location Point1', 'PHONE', 'BBL', 'BIN', 'NTA', 'Census Tract', 'Community Board', 'RECORD DATE'], axis=1, inplace=True)

In [5]:
# Check to see if data loaded properly:
rd.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,INSPECTION TYPE,Latitude,Longitude,Council District
0,50106271,SOUR MOUSE,Manhattan,110,DELANCEY STREET,10002.0,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,NaN,40.718640,-73.988481,1.0
1,50117820,MODELLO BAR - BARCLAYS CENTER,Brooklyn,620,ATLANTIC AVENUE,11217.0,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,NaN,40.683447,-73.975691,35.0
2,50141790,NaN,Manhattan,30,EAST 20 STREET,10003.0,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,NaN,40.738929,-73.989221,2.0
3,50134862,FINE AND RAW CHOCOLATE,Brooklyn,70,SCOTT AVENUE,11237.0,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,NaN,40.709853,-73.922706,34.0
4,50127238,THE DELUXE PARTY KTV,Queens,3420,LINDEN PL,11354.0,NaN,01/01/1900,NaN,NaN,NaN,Not Applicable,NaN,NaN,NaN,NaN,40.766275,-73.831338,20.0


In [6]:
rd.to_csv('rd', index=False)